# EcoTravel Agent - Demonstração Completa

Este notebook demonstra o funcionamento completo do EcoTravel Agent, integrando RAG, ferramentas customizadas e o agente conversacional.

In [ ]:
import sys
import os
import warnings
warnings.filterwarnings('ignore')

# Configurar path para imports
sys.path.append('../src')

# Imports básicos
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
from pathlib import Path
import time

# Configurar visualizações
plt.style.use('default')
sns.set_palette("viridis")
%matplotlib inline

print("🌱 EcoTravel Agent - Sistema de Viagens Sustentáveis")
print("=" * 60)

## 1. Teste das Ferramentas Individuais

### 1.1 Calculadora de Carbono

In [ ]:
try:
    from tools.carbon_calculator import CarbonCalculator
    
    # Inicializar calculadora
    calc = CarbonCalculator()
    
    print("✅ Calculadora de Carbono inicializada com sucesso!")
    
    # Teste 1: Cálculo individual
    print("\n📊 Teste 1: Cálculo de pegada de carbono individual")
    result = calc.calculate_carbon_footprint("aviao", 400, round_trip=True)
    print(f"Viagem de avião (400km ida/volta): {result['total_emission_kg_co2']:.2f} kg CO2")
    
    # Teste 2: Comparação entre modais
    print("\n📊 Teste 2: Comparação entre modais de transporte")
    comparison = calc.compare_transport_modes(400)
    
    comparison_data = []
    for comp in comparison:
        comparison_data.append({
            "Modal": comp['transport_mode'].title(),
            "Emissão (kg CO2)": round(comp['total_emission_kg_co2'], 2),
            "Fator (kg CO2/km)": round(comp['emission_per_km'], 3)
        })
    
    df_comparison = pd.DataFrame(comparison_data)
    print(df_comparison.to_string(index=False))
    
    # Teste 3: Recomendação sustentável
    print("\n📊 Teste 3: Recomendação de viagem sustentável")
    recommendation = calc.get_sustainability_recommendation(
        "São Paulo", "Rio de Janeiro", 400
    )
    
    print(f"Opção mais sustentável: {recommendation['most_sustainable_mode']['transport_mode'].title()}")
    print(f"Emissão: {recommendation['most_sustainable_mode']['total_emission_kg_co2']:.2f} kg CO2")
    print(f"Economia vs. menos sustentável: {recommendation['emission_savings_kg_co2']:.2f} kg CO2")
    print(f"Recomendação: {recommendation['recommendation']}")
    
except ImportError as e:
    print(f"❌ Erro ao importar calculadora de carbono: {e}")
    print("Certifique-se de que as dependências estão instaladas.")

### 1.2 API de Clima

In [ ]:
try:
    from tools.weather_api import WeatherAPI
    
    # Inicializar API de clima
    weather = WeatherAPI()
    
    print("✅ API de Clima inicializada com sucesso!")
    
    # Teste 1: Coordenadas de cidade
    print("\n🌤️ Teste 1: Obter coordenadas")
    coords_sp = weather.get_coordinates("São Paulo")
    coords_rj = weather.get_coordinates("Rio de Janeiro")
    
    if coords_sp:
        print(f"São Paulo: {coords_sp[0]:.2f}, {coords_sp[1]:.2f}")
    if coords_rj:
        print(f"Rio de Janeiro: {coords_rj[0]:.2f}, {coords_rj[1]:.2f}")
    
    # Teste 2: Clima atual (se coordenadas disponíveis)
    if coords_sp:
        print("\n🌤️ Teste 2: Clima atual em São Paulo")
        current = weather.get_current_weather(coords_sp[0], coords_sp[1])
        
        if current:
            print(f"Temperatura: {current.get('temperature_celsius', 'N/A')}°C")
            print(f"Sensação térmica: {current.get('feels_like_celsius', 'N/A')}°C")
            print(f"Condição: {current.get('weather_description', 'N/A')}")
            print(f"Umidade: {current.get('humidity_percent', 'N/A')}%")
    
    # Teste 3: Análise de viagem
    print("\n🌤️ Teste 3: Análise meteorológica de viagem")
    try:
        analysis = weather.analyze_travel_weather("São Paulo", "Rio de Janeiro")
        
        if "error" not in analysis:
            print("Recomendações meteorológicas:")
            for rec in analysis.get('recommendations', []):
                print(f"• {rec}")
        else:
            print(f"Erro na análise: {analysis.get('error', 'Desconhecido')}")
    except Exception as e:
        print(f"Erro na análise meteorológica: {e}")
    
except ImportError as e:
    print(f"❌ Erro ao importar API de clima: {e}")
except Exception as e:
    print(f"❌ Erro na API de clima: {e}")

### 1.3 Busca na Web

In [ ]:
try:
    from tools.web_search import WebSearchTool
    
    # Inicializar ferramenta de busca
    search = WebSearchTool()
    
    print("✅ Ferramenta de Busca Web inicializada!")
    
    # Teste 1: Busca geral
    print("\n🔍 Teste 1: Busca geral sobre turismo sustentável")
    results = search.search_web("turismo sustentável Brasil", num_results=3)
    
    if results and not results[0].get("error"):
        for i, result in enumerate(results, 1):
            print(f"{i}. {result.get('title', 'Sem título')}")
            print(f"   {result.get('snippet', 'Sem descrição')[:100]}...")
            print(f"   Fonte: {result.get('source', 'Desconhecida')}")
            print()
    else:
        print("❌ Erro na busca ou nenhum resultado encontrado")
        if results and results[0].get("error"):
            print(f"Erro: {results[0]['error']}")
    
    # Teste 2: Informações de viagem
    print("\n🔍 Teste 2: Informações específicas de viagem")
    travel_info = search.search_travel_info("Salvador", "sustentável")
    
    if travel_info.get("summary"):
        summary = travel_info["summary"]
        print(f"Categorias encontradas: {', '.join(summary.get('categories_found', []))}")
        print(f"Total de resultados: {summary.get('total_results', 0)}")
        if summary.get('key_findings'):
            print(f"Principais achados: {'; '.join(summary['key_findings'])}")
    
except ImportError as e:
    print(f"❌ Erro ao importar ferramenta de busca: {e}")
    print("A busca web pode não estar disponível sem duckduckgo-search")
except Exception as e:
    print(f"❌ Erro na busca web: {e}")

## 2. Teste do Sistema RAG

In [ ]:
try:
    from rag.rag_system import AdvancedRAGSystem
    
    print("🧠 Inicializando Sistema RAG...")
    rag_system = AdvancedRAGSystem(data_path="../data")
    
    print("📚 Construindo base de conhecimento...")
    rag_system.build_index()
    
    print("✅ Sistema RAG inicializado com sucesso!")
    print(f"📖 Documentos carregados: {len(rag_system.documents)}")
    print(f"📄 Chunks criados: {len(rag_system.chunks)}")
    
    # Teste de consultas
    test_queries = [
        "Como viajar de São Paulo para Rio de forma sustentável?",
        "Hotéis com energia solar no Brasil",
        "Emissão de carbono do transporte público"
    ]
    
    print("\n🔍 Testando consultas no sistema RAG:")
    
    for i, query in enumerate(test_queries, 1):
        print(f"\n{i}. {query}")
        print("-" * 50)
        
        results = rag_system.search(query, method="hybrid", k=2)
        
        if results:
            for j, result in enumerate(results, 1):
                score = result.get('rerank_score', result.get('score', 0))
                print(f"   Resultado {j} (Score: {score:.3f}):")
                print(f"   {result['content'][:150]}...")
                print(f"   Fonte: {result['metadata'].get('source', 'Desconhecida')}")
                print()
        else:
            print("   Nenhum resultado encontrado.")
    
    RAG_AVAILABLE = True
    
except ImportError as e:
    print(f"❌ Sistema RAG não disponível: {e}")
    print("Instale as dependências: pip install sentence-transformers faiss-cpu rank-bm25")
    RAG_AVAILABLE = False
    rag_system = None
except Exception as e:
    print(f"❌ Erro no sistema RAG: {e}")
    RAG_AVAILABLE = False
    rag_system = None

## 3. Inicialização do Agente Completo

In [ ]:
try:
    from agent.eco_travel_agent import EcoTravelAgent
    
    print("🤖 Inicializando EcoTravel Agent...")
    
    # Inicializar agente
    agent = EcoTravelAgent(
        data_path="../data",
        model_name="auto",  # Tentará Ollama, depois OpenAI, depois fallback
        verbose=True
    )
    
    print("✅ EcoTravel Agent inicializado com sucesso!")
    print(f"🛠️ Total de ferramentas disponíveis: {len(agent.tools)}")
    
    # Listar ferramentas disponíveis
    print("\n🛠️ Ferramentas disponíveis:")
    for i, tool in enumerate(agent.tools, 1):
        print(f"   {i}. {tool.name}: {tool.description}")
    
    AGENT_AVAILABLE = True
    
except ImportError as e:
    print(f"❌ Erro ao importar agente: {e}")
    AGENT_AVAILABLE = False
    agent = None
except Exception as e:
    print(f"❌ Erro ao inicializar agente: {e}")
    print("Isso pode acontecer se nenhum LLM estiver disponível (Ollama, OpenAI, etc.)")
    AGENT_AVAILABLE = False
    agent = None

## 4. Demonstração de Consultas Complexas

In [ ]:
if AGENT_AVAILABLE and agent:
    # Consultas de demonstração
    demo_queries = [
        "Quero viajar de São Paulo para o Rio de Janeiro de forma sustentável. Considere custos, emissões de carbono e condições climáticas.",
        "Preciso de recomendações de hotéis sustentáveis em Salvador com certificações ambientais.",
        "Compare as emissões de carbono entre avião, ônibus e carro para uma viagem de 600km.",
    ]
    
    print("🎯 Demonstração de Consultas Complexas")
    print("=" * 60)
    
    for i, query in enumerate(demo_queries, 1):
        print(f"\n📝 Consulta {i}: {query}")
        print("-" * 80)
        
        try:
            start_time = time.time()
            result = agent.run(query)
            execution_time = time.time() - start_time
            
            if result['success']:
                print(f"✅ Resposta (executada em {execution_time:.2f}s):")
                print(result['response'])
                
                if result['tools_used']:
                    print(f"\n🛠️ Ferramentas utilizadas: {', '.join(result['tools_used'])}")
                
                if result.get('execution_steps'):
                    print(f"\n📊 Passos executados: {len(result['execution_steps'])}")
            else:
                print(f"❌ Erro na execução: {result.get('error', 'Erro desconhecido')}")
                
        except Exception as e:
            print(f"❌ Erro durante execução: {e}")
        
        print("\n" + "="*80)
        
        # Pausa entre consultas para não sobrecarregar
        if i < len(demo_queries):
            time.sleep(2)

else:
    print("❌ Agente não disponível para demonstração de consultas complexas.")
    print("\n🔧 Para usar o agente completo, você precisa:")
    print("1. Instalar Ollama e baixar um modelo (ex: llama3)")
    print("2. OU configurar OpenAI API key")
    print("3. OU instalar transformers para modelos locais")
    print("\n💡 Demonstração das ferramentas individuais funcionando acima!")

## 5. Análise de Sustentabilidade de Planos de Viagem

In [ ]:
if AGENT_AVAILABLE and agent:
    print("🌱 Análise de Sustentabilidade de Planos de Viagem")
    print("=" * 60)
    
    # Criar planos de viagem de exemplo para análise
    sample_plans = [
        {
            "name": "Viagem Sustentável SP-RJ",
            "transport_mode": "onibus",
            "distance_km": 400,
            "duration_days": 3,
            "eco_hotel": True,
            "local_activities": True,
            "passengers": 2
        },
        {
            "name": "Viagem Convencional SP-RJ",
            "transport_mode": "aviao",
            "distance_km": 400,
            "duration_days": 2,
            "eco_hotel": False,
            "local_activities": False,
            "passengers": 1
        },
        {
            "name": "Viagem Mista SP-Salvador",
            "transport_mode": "aviao",
            "distance_km": 1200,
            "duration_days": 7,
            "eco_hotel": True,
            "local_activities": True,
            "passengers": 1
        }
    ]
    
    sustainability_results = []
    
    for plan in sample_plans:
        print(f"\n📋 Analisando: {plan['name']}")
        
        score_result = agent.get_sustainability_score(plan)
        
        print(f"🌱 Score de Sustentabilidade: {score_result['score']}/100")
        print(f"📊 Categoria: {score_result['category']}")
        print(f"✅ Fatores positivos: {', '.join(score_result['factors'])}")
        
        if score_result['recommendations']:
            print("💡 Recomendações para melhorar:")
            for rec in score_result['recommendations']:
                print(f"   • {rec}")
        
        sustainability_results.append({
            "Plano": plan['name'],
            "Score": score_result['score'],
            "Categoria": score_result['category'],
            "Transporte": plan['transport_mode'].title(),
            "Distância": plan['distance_km']
        })
    
    # Visualizar comparação
    df_sustainability = pd.DataFrame(sustainability_results)
    
    plt.figure(figsize=(12, 6))
    
    # Gráfico de barras dos scores
    plt.subplot(1, 2, 1)
    colors = ['green' if score >= 70 else 'orange' if score >= 50 else 'red' 
              for score in df_sustainability['Score']]
    bars = plt.bar(range(len(df_sustainability)), df_sustainability['Score'], color=colors, alpha=0.7)
    plt.title('Scores de Sustentabilidade dos Planos')
    plt.ylabel('Score (0-100)')
    plt.xticks(range(len(df_sustainability)), [f"Plano {i+1}" for i in range(len(df_sustainability))])
    
    # Adicionar valores nas barras
    for i, bar in enumerate(bars):
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 1,
                 f'{int(height)}', ha='center', va='bottom')
    
    # Gráfico de dispersão Score vs Distância
    plt.subplot(1, 2, 2)
    transport_colors = {'onibus': 'green', 'aviao': 'red', 'carro': 'orange'}
    for transport in df_sustainability['Transporte'].unique():
        subset = df_sustainability[df_sustainability['Transporte'] == transport.title()]
        plt.scatter(subset['Distância'], subset['Score'], 
                   label=transport.title(), 
                   color=transport_colors.get(transport.lower(), 'blue'), 
                   s=100, alpha=0.7)
    
    plt.title('Score vs Distância por Modal')
    plt.xlabel('Distância (km)')
    plt.ylabel('Score de Sustentabilidade')
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    
    print("\n📊 Resumo da Análise:")
    print(df_sustainability.to_string(index=False))
    
else:
    print("❌ Análise de sustentabilidade não disponível sem o agente.")

## 6. Interface de Chat Interativo (Simulação)

In [ ]:
# Esta célula simula um chat interativo
# Para usar realmente, execute: agent.chat() em um ambiente Python normal

print("💬 Simulação de Interface de Chat")
print("=" * 50)
print()
print("🤖 EcoTravel Agent: Olá! Sou seu assistente de viagens sustentáveis.")
print("Como posso ajudá-lo a planejar uma viagem mais ecológica?")
print()
print("👤 Usuário: Quero viajar para Fernando de Noronha de forma sustentável")
print()

if AGENT_AVAILABLE and agent:
    # Simular resposta do agente
    sample_query = "Como viajar para Fernando de Noronha de forma sustentável?"
    
    try:
        print("🤖 EcoTravel Agent: Analisando sua solicitação...")
        result = agent.run(sample_query)
        
        if result['success']:
            print(f"🤖 EcoTravel Agent: {result['response']}")
        else:
            print(f"🤖 EcoTravel Agent: Desculpe, tive um problema: {result.get('error', 'Erro desconhecido')}")
            
    except Exception as e:
        print(f"🤖 EcoTravel Agent: Houve um erro: {e}")
else:
    print("🤖 EcoTravel Agent: Para Fernando de Noronha, recomendo:")
    print("• Voo até Recife, depois voo regional (única opção de acesso)")
    print("• Hospede-se em pousadas com certificação ambiental")
    print("• Participe apenas de atividades autorizadas pelo ICMBio")
    print("• Contribua com a taxa de preservação ambiental")
    print("• Evite usar produtos não biodegradáveis na ilha")

print()
print("💡 Para iniciar um chat interativo real, execute:")
print("   agent.chat()")
print("   em um ambiente Python normal (não Jupyter)")

## 7. Métricas e Estatísticas do Sistema

In [ ]:
print("📊 Métricas e Estatísticas do Sistema")
print("=" * 50)

# Métricas dos dados
print("\n📚 Base de Conhecimento:")
print(f"   • Guias de viagem: {len(list(Path('../data/guias').glob('*.txt')))} arquivos")
print(f"   • Dados de emissões: {len(list(Path('../data/emissoes').glob('*.csv')))} arquivos")
print(f"   • Avaliações de hotéis: {len(list(Path('../data/avaliacoes').glob('*.json')))} arquivos")

# Estatísticas dos dados de emissões
try:
    emissoes_df = pd.read_csv('../data/emissoes/emissoes_transporte.csv')
    print(f"\n🚗 Dados de Transporte:")
    print(f"   • Modais de transporte: {len(emissoes_df)} tipos")
    print(f"   • Menor emissão: {emissoes_df['emissao_co2_kg_km'].min():.3f} kg CO2/km ({emissoes_df.loc[emissoes_df['emissao_co2_kg_km'].idxmin(), 'modal_transporte']})")
    print(f"   • Maior emissão: {emissoes_df['emissao_co2_kg_km'].max():.3f} kg CO2/km ({emissoes_df.loc[emissoes_df['emissao_co2_kg_km'].idxmax(), 'modal_transporte']})")
    print(f"   • Emissão média: {emissoes_df['emissao_co2_kg_km'].mean():.3f} kg CO2/km")
except:
    print("   ❌ Erro ao carregar dados de emissões")

# Estatísticas dos hotéis
try:
    with open('../data/avaliacoes/hoteis_sustentaveis.json', 'r', encoding='utf-8') as f:
        hoteis_data = json.load(f)
    
    hoteis_df = pd.DataFrame(hoteis_data['hotéis_sustentáveis'])
    print(f"\n🏨 Hotéis Sustentáveis:")
    print(f"   • Total de hotéis: {len(hoteis_df)}")
    print(f"   • Estados cobertos: {hoteis_df['estado'].nunique()}")
    print(f"   • Nota média de sustentabilidade: {hoteis_df['nota_sustentabilidade'].mean():.1f}/10")
    print(f"   • Preço médio da diária: R$ {hoteis_df['preço_diária'].mean():.0f}")
except:
    print("   ❌ Erro ao carregar dados de hotéis")

# Métricas do sistema RAG
if RAG_AVAILABLE and rag_system:
    print(f"\n🧠 Sistema RAG:")
    print(f"   • Documentos processados: {len(rag_system.documents)}")
    print(f"   • Chunks criados: {len(rag_system.chunks)}")
    if hasattr(rag_system, 'embeddings') and rag_system.embeddings is not None:
        print(f"   • Dimensão dos embeddings: {rag_system.embeddings.shape[1]}")
        print(f"   • Total de vetores: {rag_system.embeddings.shape[0]}")

# Métricas do agente
if AGENT_AVAILABLE and agent:
    print(f"\n🤖 Agente EcoTravel:")
    print(f"   • Ferramentas disponíveis: {len(agent.tools)}")
    print(f"   • Tipos de ferramentas:")
    
    tool_types = {}
    for tool in agent.tools:
        if "Carbon" in tool.name:
            tool_types["Carbono"] = tool_types.get("Carbono", 0) + 1
        elif "Weather" in tool.name or "Climate" in tool.name:
            tool_types["Clima"] = tool_types.get("Clima", 0) + 1
        elif "Search" in tool.name or "Web" in tool.name:
            tool_types["Busca"] = tool_types.get("Busca", 0) + 1
        elif "Knowledge" in tool.name or "RAG" in tool.name:
            tool_types["Conhecimento"] = tool_types.get("Conhecimento", 0) + 1
        else:
            tool_types["Outros"] = tool_types.get("Outros", 0) + 1
    
    for tool_type, count in tool_types.items():
        print(f"     - {tool_type}: {count} ferramentas")

# Status geral do sistema
print(f"\n✅ Status do Sistema:")
print(f"   • Calculadora de Carbono: {'✅ Funcionando' if 'calc' in locals() else '❌ Indisponível'}")
print(f"   • API de Clima: {'✅ Funcionando' if 'weather' in locals() else '❌ Indisponível'}")
print(f"   • Busca Web: {'✅ Funcionando' if 'search' in locals() else '❌ Indisponível'}")
print(f"   • Sistema RAG: {'✅ Funcionando' if RAG_AVAILABLE else '❌ Indisponível'}")
print(f"   • Agente Completo: {'✅ Funcionando' if AGENT_AVAILABLE else '❌ Indisponível'}")

# Calcular economia potencial de CO2
if 'calc' in locals():
    print("\n🌱 Potencial de Economia de CO2:")
    
    # Exemplos de rotas populares
    routes = [
        ("SP-RJ", 400),
        ("SP-BSB", 1000),
        ("RJ-SSA", 1200),
        ("SP-Floripa", 600)
    ]
    
    total_savings = 0
    for route_name, distance in routes:
        comparison = calc.compare_transport_modes(distance)
        savings = comparison[-1]['total_emission_kg_co2'] - comparison[0]['total_emission_kg_co2']
        total_savings += savings
        print(f"   • {route_name} ({distance}km): até {savings:.1f} kg CO2 economizados")
    
    print(f"   • Economia média por rota: {total_savings/len(routes):.1f} kg CO2")
    print(f"   • Equivalent a plantar ~{int(total_savings/len(routes)/22)} árvores por viagem")

## 8. Conclusões e Próximos Passos

### 🎯 Objetivos Alcançados:

1. **✅ Sistema RAG Funcional**: Base de conhecimento sobre viagens sustentáveis com busca híbrida
2. **✅ Ferramentas Especializadas**: Cálculo de carbono, API de clima, busca web
3. **✅ Agente Inteligente**: Orquestração de múltiplas ferramentas com padrão ReAct
4. **✅ Análise de Sustentabilidade**: Sistema de scoring para planos de viagem
5. **✅ Interface Amigável**: Chat interativo e análises visuais

### 📊 Principais Características:

- **RAG Avançado**: Busca semântica + lexical com reranking
- **Cálculos Precisos**: Emissões de CO2 por modal de transporte
- **Informações Atualizadas**: APIs de clima e busca web
- **Recomendações Personalizadas**: Baseadas em sustentabilidade
- **Métricas Quantificáveis**: Economia de CO2, scores de sustentabilidade

### 🚀 Próximos Passos:

1. **Expansão da Base de Conhecimento**:
   - Mais guias de destinos
   - Dados de transporte público
   - Informações de eventos sustentáveis

2. **Melhorias no Agente**:
   - Fine-tuning do modelo
   - Feedback loop dos usuários
   - Integração com APIs de reservas

3. **Interface Avançada**:
   - WebApp com Streamlit/Gradio
   - Visualizações interativas
   - Sistema de recomendações proativas

4. **Funcionalidades Adicionais**:
   - Compensação automática de carbono
   - Integração com calendários
   - Alertas de sustentabilidade

### 🌱 Impacto Esperado:

- **Redução de Emissões**: Até 50-80% por viagem escolhendo modais sustentáveis
- **Consciência Ambiental**: Educação sobre pegada de carbono
- **Turismo Responsável**: Apoio a negócios locais sustentáveis
- **Decisões Informadas**: Dados precisos para escolhas conscientes

---

### 💡 Para Experimentar:

```python
# Em um ambiente Python normal (não Jupyter):
from src.agent.eco_travel_agent import EcoTravelAgent

agent = EcoTravelAgent()
agent.chat()  # Inicia chat interativo
```

### 🔧 Dependências Opcionais:

```bash
# Para funcionalidade completa:
pip install ollama  # OU configure OPENAI_API_KEY
pip install duckduckgo-search  # Para busca web
```

**🌍 EcoTravel Agent: Transformando a forma como planejamos viagens sustentáveis!**